# Laboratory 2.6: LOO + k-Fold Cross Validation

In this practice we will implement one of the main techniques to prevent overfitting when training a model: **cross-validation**.

In addition, we will be using the following libraries:
- Data management:
    - [numpy](https://numpy.org/)
- Modelling and scoring:
    - [scikit-learn](https://scikit-learn.org)
- Plotting:
    - [matplotlib](https://matplotlib.org/)
    
### **All the things you need to do are marked by a "TODO" comment nearby. Make sure you *read carefully everything before working* and solve each point before submitting your solution.**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os
import sys
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path
sys.path.insert(0, project_root)

In the following cell import the training (`training.dat`) and test (`test.dat`) dataset. We recommend you to use the `np.loadtxt()` function.

You will need to create the `X_train`, `y_train`, `X_test` and `y_test` variables. Take into account that each dataset have 10 variables, where the last one is the output variable.

In [2]:
# TODO: Load training.dat and text.dat and create X_train, y_train, X_test and y_test
train = np.loadtxt('training.dat',delimiter=',')
X_train = train[:,0:10]
y_train = train[:,10]
test = np.loadtxt('test.dat',delimiter=',')
X_test = test[:,0:10]
y_test = test[:,10]

With this data you are going to train the optimal `KNeighborsClassifier` model.

### Initial guess

As you have no idea of what the optimal value of `n_neighbors` is, trust your professors and use `n_neighbors = 4` to train your model.

In [3]:
# TODO: Train a KNeighborsClassifier model with n_neighbors=4
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

Now, calculate the accuracy of the model for the training and test sets using the `accuracy_score` function from `scikit-learn`.

In [4]:
# TODO: Calculate accuracy in training and test for KNN with k=4
acc_train = accuracy_score(y_train,model.predict(X_train))
acc_test = accuracy_score(y_test,model.predict(X_test))
print(f'Accuracy, train = {acc_train} test = {acc_test}')

Accuracy, train = 0.84375 test = 0.785


**What is happening with this value of `n_neighbors`?**
> The model performs a lot better with training data which is to be expected given in was trained with it, but given the difference with the acuracy of the test set the model might be overfitting. For test data it does worse  with an acuracy of 0.785 which means that 78.5% of the data where clasified correctly, which we would ideally want to be higher.

### Damage control
It seems that `n_neighbors = 4` overfits for this dataset. Let's try to correct this and use `n_neighbors = 20`

In [5]:
# TODO: Train a KNeighborsClassifier model with n_neighbors=20
model = KNeighborsClassifier(n_neighbors=20)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

Calculate again the accuracy of training and test sets for this model

In [6]:
# TODO: Calculate accuracy in training and test for KNN with k=20
acc_train = accuracy_score(y_train,model.predict(X_train))
acc_test = accuracy_score(y_test,model.predict(X_test))
print(f'Accuracy, train = {acc_train} test = {acc_test}')

Accuracy, train = 0.81 test = 0.795


It seems that the accuracy of the test set has improved, but how can we be sure that `n_neighbors=20` is the optimal value?

### Obtaining optimal value for hyperparameters

We could keep trying with different values for `n_neighbors` until we find the optimal one. However, this strategy is unfeasible for real datasets. So how can we obtain a reasonable optimal value for the hyperparameters of a model?

We can leverage the power of cross-validation:

<center> <img title="5-Fold Cross-Validation " alt="cross-validation" src="https://miro.medium.com/v2/resize:fit:1200/1*AAwIlHM8TpAVe4l2FihNUQ.png"> </center>

Cross-validation give us a notion of the generalization error (i.e., test error) using parts of the training set as a validation set. This validation set is data that the trained model has not seen before, so if the model performs well in this part of the dataset it should generalize well in unseen data. However, as a fortunate partition might happen and generalization error might be underestimated, instead of using a single validation set we use K validation sets and we use the mean error in this K sets as the CV-error. This CV-error should give us a reliable estimation of the generalization error.

But, how can we use it to obtain the optimal hyperparameters of the model? If the CV-error is an estimation of the generalization error, the hyperparameter values with least CV-error would result in the least generalization error. As CV-error can be computed during training, we obtain a faster way to obtain the optimal values of the hyperparameters.

Now that we know why we want cross validation, implement the `cross_validation` function. This function shall implement two cross validation methods:
- K-Fold cross validation
- Leave-one-out cross validation

Check this [link](https://machinelearningmastery.com/k-fold-cross-validation/) to know the details of each method.

In [7]:
from src.Lab2_6_CV import cross_validation

In [8]:
cross_validation(model,X_train,y_train,5)

(np.float64(0.7887500000000001), np.float64(0.023518609652783476))

Now that we have the `cross_validation` function implemented, let's check which is the optimal value for `n_neighbors` in this problem.

In [ ]:
# Initialize lists to store mean scores and standard deviations for each value of k
mean_scores = []
std_scores = []

# Define the range of k values to test
k_values = range(4, 80)

# TODO: Loop through each value of k, obtaining the mean accuracy score and the standard deviation of the accuracy score in cross validation
results=[]
for k in k_values:
    model=KNeighborsClassifier(n_neighbors=k)
    mean,std=cross_validation(model,X_train,y_train,5)
    mean_scores.append(mean)
    std_scores.append(std)


# # TODO: Find the highest score and the corresponding optimal k
# highest_score = 
# optimal_k =

# print(f"Optimal value of k: {optimal_k} with a score of {highest_score:.2f}")

# # Plotting
# plt.figure(figsize=(10, 6))
# plt.errorbar(k_values, mean_scores, yerr=std_scores, fmt='-o', ecolor='r', capsize=5, capthick=2, markersize=5, label='CV Score +/- std dev')
# plt.axvline(x=optimal_k, linestyle='--', color='k', label=f'Optimal k: {optimal_k}')

# plt.title('kNN Model Complexity: Cross-Validation Scores')
# plt.xlabel('Number of Neighbors (k)')
# plt.ylabel('CV Mean Score')
# plt.legend()
# plt.grid(True)
# plt.show()


(np.float64(0.7612500000000001), np.float64(0.025739075352467493))
(np.float64(0.805), np.float64(0.016955824957813188))
(np.float64(0.7837500000000001), np.float64(0.010897247358851678))
(np.float64(0.805), np.float64(0.019921721813136516))
(np.float64(0.7887500000000001), np.float64(0.017853571071357138))
(np.float64(0.81125), np.float64(0.019921721813136527))
(np.float64(0.8012499999999999), np.float64(0.01075290658380329))
(np.float64(0.805), np.float64(0.01912132317597292))
(np.float64(0.7925), np.float64(0.015512092057488564))
(np.float64(0.7975), np.float64(0.014031215200402278))
(np.float64(0.7925), np.float64(0.014469796128487781))
(np.float64(0.79125), np.float64(0.02325134404717284))
(np.float64(0.79375), np.float64(0.020916500663351868))
(np.float64(0.7975), np.float64(0.01960548392669767))
(np.float64(0.7925), np.float64(0.02417384950726714))
(np.float64(0.79625), np.float64(0.02455860338048562))
(np.float64(0.7887500000000001), np.float64(0.023518609652783476))
(np.float6

Now that we know the optimal value for `n_neighbors`, let's train a KNN model with this value of hyperparameter and check if the generalization error has improved.

In [ ]:
# TODO: Train model with k=optimal_k
model = None
model.fit(X_train, y_train)

In [ ]:
# TODO: Calculate accuracy in training and test for KNN with k=optimal_k
acc_train = None
acc_test = None
print(f'Accuracy, train = {acc_train} test = {acc_test}')

### Sensitivity analysis
- Does the number of folds affects the optimal value of `n_neighbors`? Why or why not?
> Write your answer here

- What happens with the computational time if you increment the number of folds?
> Write your answer here

- Does it worth to increase the number of folds? Is the CV error a better proxy of the test set?
> Write your answer here